### Comparison between the A/E method and the hybrid approach autoencoder+A/E+classifier
The hybrid approach consists of training the classifier NN with both deep encoded and A/E features.

In [1]:
from begepro.autoencoder.sn_analysis import Comparison
from begepro.autoencoder.dataloading import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from begepro.autoencoder.autoencoder_classifier import Autoencoder, Classifier


2023-02-10 14:54:27.259887: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 14:54:27.436772: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-10 14:54:27.436793: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-10 14:54:28.033112: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
opts = {"loadpath": "/mnt/e/BEGE_data/waveforms_crioconite", # where .bin files are stored
"savepath": "../../dataset/Th232", # where .npy files will be saved with readfiles() and loaded with load()
"subsampling_factor": 3}

dloader = DataLoader(opts)
waveforms, _, energies, _, _ = dloader.load()

opts_2 = {"loadpath": "/mnt/e/BEGE_data/waveforms_crioconite", # where .bin files are stored
"savepath": "../../dataset/Th232_2", # where .npy files will be saved with readfiles() and loaded with load()
"subsampling_factor": 3}

dloader_2 = DataLoader(opts_2)
waveforms_2, _, energies_2, _, _ = dloader_2.load()

In [3]:
preds = np.load("../../preds_ratios/preds_hybrid.npy")[:,0]
ratio = np.load("../../preds_ratios/ratio.npy")
ratio_2 = np.load("../../preds_ratios/ratio_2.npy")

waveforms = np.concatenate([waveforms, waveforms_2])
ratio = np.concatenate([ratio, ratio_2])
energies = np.concatenate([energies, energies_2])

In [4]:
comparison = Comparison(energies, ratio, preds)
M_ELECTRON = 511
cs = np.linspace(0.0, 0.7, 70)
peaks = [2615, 2615 - 1*M_ELECTRON, 2615 - 2*M_ELECTRON, 1588, 1622, 1460, 860, 835, 409, 583, 2177]

In [5]:
sb_nn, std_sb_nn, sb_avse, std_sb_avse, avse_thresholds = comparison.compare(peaks, cs, benchmark = "nn")

Estimating S/B for peak at 2615 keV
Estimating S/B for peak at 2104 keV
Estimating S/B for peak at 1593 keV
Estimating S/B for peak at 1588 keV
Estimating S/B for peak at 1622 keV
Estimating S/B for peak at 1460 keV
Estimating S/B for peak at 860 keV
Estimating S/B for peak at 835 keV
Estimating S/B for peak at 409 keV
Estimating S/B for peak at 583 keV
Estimating S/B for peak at 2177 keV


/home/rmoretti/BEGe/BEGepro/begepro/autoencoder/sn_analysis.py:110: UserWarning: The fit did not converge.
  warnings.warn("The fit did not converge.")


In [6]:
sb_nn[65,10] = None

In [8]:
plt.rc('xtick', labelsize=8) 
for n in range(len(peaks)):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax2 = ax1.twiny()

    ax1.plot(cs, sb_nn[:,n], color =  'deepskyblue')
    ax1.fill_between(cs, sb_nn[:,n] + std_sb_nn[:,n], sb_nn[:,n] - std_sb_nn[:,n],label='_nolegend_', alpha = 0.5, color = 'skyblue')
    ax1.plot(cs, sb_avse[:,n], color = 'red')
    ax1.fill_between(cs, sb_avse[:,n] + std_sb_avse[:,n], sb_avse[:,n] - std_sb_avse[:,n], color = 'coral', label='_nolegend_', alpha = 0.5)

    d = 10
    new_tick_locations = np.array(cs[::d])

    ax1.set_xticks(cs[::d])

    ax1.set_xticklabels(["> "+str(round(float(c), 2)) for c in cs[::d]])

    ax2.set_xlim(ax1.get_xlim())
    ax2.set_xticks(cs[::d])
    ax2.set_xticklabels(["< "+str(round(float(avse_threshold*100), 1)) for avse_threshold in avse_thresholds[::d,n]])

    ax1.set_xlabel("NN score threshold")
    ax2.set_xlabel("A/E score threshold [x 100]")
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")
    ax2.spines["bottom"].set_position(("axes", -0.15))

    ax1.set_ylabel("S/B ratio")
    plt.title(f"Peak at {peaks[n]} keV")
    ax1.legend(["NN", "A/E"])
    plt.savefig(f"../../results/z_peak_at_{peaks[n]}_122_sampling_hybrid.svg", bbox_inches='tight')
    plt.clf()


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>